<a href="https://colab.research.google.com/github/ShraddhaSharma24/Deep-Learning/blob/main/ECG_Based_Heart_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install wfdb numpy pandas matplotlib scikit-learn

In [2]:
import numpy as np
import wfdb
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Download MIT-BIH Arrhythmia Database
!wget -r -N -c -np https://physionet.org/files/mitdb/1.0.0/ -P /content/mitdb/
!find /content/mitdb/ -name "*.hea"

# Function to load ECG signals from local storage
def load_ecg_data(record_ids):
    data, labels = [], []

    for rec in record_ids:
        record = wfdb.rdrecord(f'/content/mitdb/physionet.org/files/mitdb/1.0.0/{rec}', channels=[0])
        annotation = wfdb.rdann(f'/content/mitdb/physionet.org/files/mitdb/1.0.0/{rec}', 'atr')

        signal = record.p_signal.flatten()  # Convert to 1D array
        data.append(signal)

        # Assign labels (Normal: 0, Abnormal: 1)
        label = 0 if rec in ['100', '101', '102', '103', '104'] else 1  # Example labeling
        labels.append(label)

    return np.array(data), np.array(labels)



--2025-03-08 12:18:27--  https://physionet.org/files/mitdb/1.0.0/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/mitdb/physionet.org/files/mitdb/1.0.0/index.html’

physionet.org/files     [ <=>                ]  22.86K  --.-KB/s    in 0.01s   

Last-modified header missing -- time-stamps turned off.
2025-03-08 12:18:28 (1.94 MB/s) - ‘/content/mitdb/physionet.org/files/mitdb/1.0.0/index.html’ saved [23410]

Loading robots.txt; please ignore errors.
--2025-03-08 12:18:28--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.

--2025-03-08 12:18:28--  https://physionet.org/files/mitdb/1.0.0/mitdbdir/
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting

In [3]:
# Load ECG data after downloading
record_ids = ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109']
X, y = load_ecg_data(record_ids)


In [4]:
# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape for CNN-LSTM input (samples, timesteps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)




In [5]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [6]:
# CNN-LSTM Model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))



Epoch 1/10


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")